# Capstone Project on TED talk analysis : Data Wrangling

Springboard Data Science Career Track ; 
Author : Pavan Poosarla, pavanpoosarla01@gmail.com

Start Date : 9/ 10/ 2019
Description :
As a part of the first capstone project, I will be analysing TED talk transcripts and analyse the sentiment

Date Source
https://www.kaggle.com/rounakbanik/ted-talks/downloads/ted-talks.zip/3

## Part 1 : Data Wrangling

In [1]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import string
import nltk
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')
from tqdm import tqdm
import json


[nltk_data] Downloading package punkt to C:\Users\Pavan
[nltk_data]     Anirudh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Define function to check for missing values
def CheckMissing_df(df):
    '''Function to remove missing values from the dataframe. It does the following
    Print the number of missing values in each columns
    '''
    missing_dict = {'ColName' : [], 'MissingNumber':[]}
    df_cols = df.columns
    for item in df_cols:
        missing_dict['ColName'].append(item)
        missing_dict['MissingNumber'].append(df[item].isna().sum())
    missing_df = pd.DataFrame(missing_dict)
    return missing_df

In [3]:
# Define function to count the number of sentences in a text
def CountSentence(text):
    '''Fuction to count the number of sentences in a text in a dataframe'''
    sen_list = text.split('.')
    sen_list = [x for x in sen_list if x != '']
    return len(sen_list)

In [4]:
def AudReaction(text):
    '''Function to extract all audience reactions provided within the transcripts as a list'''
    openn = text.split('(')
    aud_react_list = []
    for txt in openn:
        closee = txt.split(')')
        if len(closee)==2:
            aud_react_list.append(closee[0])
    aud_react_count = len(aud_react_list)
    aud_react_dict = dict((x,aud_react_list.count(x)) for x in set(aud_react_list))
    return aud_react_dict   

In [5]:
def WordCount (text):
    '''Function to extract word count from a transcript after eliminating text in parenthesis'''
    sen_list = text.split('.')
    sen_list = [x for x in sen_list if x != '']
    len(sen_list)
    clean_sen = []
    for sen in sen_list:
        openn = sen.split('(')
        if len(openn) == 1:
            clean_sen.append(sen)
        else:
            for op in openn:
                closee = op.split(')')
                if len(closee) == 2:
                    closee = closee[1]
                elif len(closee) == 1:
                    closee = closee[0]
                else:
                    # split_2way = op.split(')',2)  
                    # closee = split_2way[1]
                    closee = ')'.join(closee)
                    # print ('error:', op)
                    # print (closee)
            clean_sen.append(closee)
        # print (sen)
        # print (close)
    word_count = sum([len(x.split(' ')) for x in clean_sen])
    return word_count

In [6]:
def RatingsDict (text):
    import json
    str_list = text.strip('[]').split('},')
    new_list = []
    for X in str_list:
        if X[-1] != '}':
            new_list.append(X+'}')
        else:
            new_list.append(X)
    # new_list = [(X+'}') for X in new_list if X[-1] != '}']
    new_list = [X.replace('\'', '\"') for X in new_list]
    dict_list = [json.loads(X) for X in new_list]
    # dict_list
    
    rating_dict = {}
    for item in dict_list:
        rating_dict[item['name']] =  item['count']
    
    return rating_dict

In [7]:
# Functon for text normalization
def text_normalize(text):
    normalized_text = text
    # Strip leading and lagging whitespace
    normalized_text = normalized_text.strip()
    # Convert all text to lower case
    normalized_text.lower()
    # Remove punctuation
    normalized_text.translate( str.maketrans('','', string.punctuation))
    # Word tokenization
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(normalized_text)
    result = [i for i in tokens if not i in stop_words]
    stemmer = PorterStemmer()
    stemmed_result = []
    for word in result:
        stemmed_result.append(stemmer.stem(word))
    return stemmed_result

In [8]:
# Function to reduce the number of unique entries in event
def ted_eventtype(text):
    if "TED" in text:
        if "TEDx" in text:
            return "TEDx"
        if "TEDMED" in text:
            return "TEDMED"
        if "TEDGlobal" in text:
            return "TEDGlobal"
        if "TEDWomen" in text:
            return "TEDWomen"
        if "TEDYouth" in text:
            return "TEDYouth"
        if "TEDSalon" in text:
            return "TEDSalon"
        if "TEDActive" in text:
            return "TEDActive"
        return "TED"
    else:
        return "Other"
    

In [9]:
# Read data from the Kaggle TED talk database
df_main = pd.read_csv(r'../data/raw/ted_main.csv', parse_dates = ['film_date', 'published_date'])
df_transcripts = pd.read_csv(r'../data/raw/transcripts.csv')

In [10]:
df_main.head(2)

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520


In [11]:
df_main.shape

(2550, 17)

In [12]:
df_transcripts.shape

(2467, 2)

In [13]:
df_transcripts.head(2)

,transcript,url
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...


In [14]:
# Join both databases
# lets work with those combined dataframe from both files
df_joined = pd.merge (df_main, df_transcripts, how = 'inner', on = 'url')
print ('Combined Dataframe is read')
print ('Raw dataset has %d columns and %d talks'%(df_joined.shape[1], df_joined.shape[0]))

Combined Dataframe is read
Raw dataset has 18 columns and 2467 talks


In [15]:
# Look for any missing data
missing_df = CheckMissing_df(df_joined)
print ('Columns with missing values are \n', missing_df[missing_df.MissingNumber != 0])

Columns with missing values are 
                ColName  MissingNumber
12  speaker_occupation              6


In [16]:
# Looks like speaker occupation is not available for 6 talks
# Fill Missing values
df_joined['speaker_occupation'].fillna('Unknown', inplace = True)
print ('After filling, # missing in speaker occupation is', df_joined['speaker_occupation'].isna().sum())

After filling, # missing in speaker occupation is 0


## Building a clean dataset

In [17]:
df_clean = df_joined

In [18]:
# Drop talks with more than 1 speaker
df_clean = df_joined[df_joined.num_speaker == 1] # To avoid convoluting the analysis

In [19]:
# Drop the columns that are not meaningful to the analysis
df_clean = df_clean.drop(columns = ['related_talks', 'languages', 'url','name','views', 'num_speaker'])
print ('After dropping extra columns, dataset has', df_clean.columns)

After dropping extra columns, dataset has Index(['comments', 'description', 'duration', 'event', 'film_date',
       'main_speaker', 'published_date', 'ratings', 'speaker_occupation',
       'tags', 'title', 'transcript'],
      dtype='object')


In [20]:
df_clean.dtypes

comments               int64
description           object
duration               int64
event                 object
film_date             object
main_speaker          object
published_date        object
ratings               object
speaker_occupation    object
tags                  object
title                 object
transcript            object
dtype: object

In [21]:
# Convert film_date into human readable format
df_clean['film_datestamp'] = pd.to_datetime(df_clean['film_date'],unit='s')
df_clean.film_datestamp.head(3)

0   2006-02-25
1   2006-02-25
2   2006-02-24
Name: film_datestamp, dtype: datetime64[ns]

In [22]:
# Do the same date conversaion with published datetime
df_clean['pub_datestamp'] = pd.to_datetime(df_clean['published_date'], unit = 's')
df_clean.pub_datestamp.head(3)

0   2006-06-27 00:11:00
1   2006-06-27 00:11:00
2   2006-06-27 00:11:00
Name: pub_datestamp, dtype: datetime64[ns]

# Text Analysis

1. Sentence Counter
Defining the number of sentence talks in spoken english needs some thought. Turns out, we do not speak in perfecty grammmatical sentences. So, we basically look at number of pauses in the talk to get an estimation of number of sentences. While this choice may be deatable from a purely grammatical standpoint, it lets us know where the speaker paused. As that may make the speech understandable, we will just count in thus way. 


In [23]:
# Build a sentence counter
df_clean['sentence_count'] = df_clean['transcript'].apply(CountSentence)

In [24]:
# Build a dictionary of audience reaction throughout the talk
df_clean['aud_reaction_dict'] = df_clean['transcript'].apply(AudReaction)


In [25]:
# Lets build a word counter
df_clean['word_count'] = df_clean['transcript'].apply(WordCount)
# df_clean

In [26]:
df_clean.head(3)

,comments,description,duration,event,film_date,main_speaker,published_date,ratings,speaker_occupation,tags,title,transcript,film_datestamp,pub_datestamp,sentence_count,aud_reaction_dict,word_count
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,Ken Robinson,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,Good morning. How are you?(Laughter)It's been ...,2006-02-25,2006-06-27 00:11:00,223,"{'Laughter': 39, 'Applause': 4}",3172
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,Al Gore,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,"Thank you so much, Chris. And it's truly a gre...",2006-02-25,2006-06-27 00:11:00,141,"{'Mock sob': 2, 'Laughter': 22, 'Applause': 6}",2081
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,David Pogue,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,"(Music: ""The Sound of Silence,"" Simon & Garfun...",2006-02-24,2006-06-27 00:11:00,250,"{'Music: ""The Sound of Silence,"" Simon & Garfu...",3375


In [27]:
df_clean.dtypes

comments                       int64
description                   object
duration                       int64
event                         object
film_date                     object
main_speaker                  object
published_date                object
ratings                       object
speaker_occupation            object
tags                          object
title                         object
transcript                    object
film_datestamp        datetime64[ns]
pub_datestamp         datetime64[ns]
sentence_count                 int64
aud_reaction_dict             object
word_count                     int64
dtype: object

## Creating a Audience reaction Columns
To create audience reaction columns, we do the following
1. Extract audience reactions from transcript and have them as a dictionary
2. Convert the dictionary to a dataframe, resulting in a sparsely populated df with several columns
3. Consolidate this df by merging sparse columns and filling in misisng values
4. Reduce number of columns in this way and append to clean dataframe

In [28]:
# Example of dict element in audience reaction column
df_clean.loc[2,'aud_reaction_dict']

{'Music: "The Sound of Silence," Simon & Garfunkel': 1,
 'Laughter': 38,
 'Piano music': 1,
 'Applause': 12,
 'Music': 2}

In [29]:
# Convert the dictionary of audience reactions into separate columns for aud reactions
df_aud_react = df_clean['aud_reaction_dict'].apply(pd.Series)
df_aud_react.head(3)

,Laughter,Applause,Mock sob,"Music: ""The Sound of Silence,"" Simon & Garfunkel",Piano music,Music,Audience whistles,Applause ends,Laughter and applause,Applause continues,...,Distorted voice,Speaks more loudly,Khmer,Singing in Arabic,Cracking sound,Audience guesses,Audience murmurs,Audience responds,A capella singing,offscreen
0,39.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38.0,12.0,NaN,1.0,1.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As can be seen, there are 791 unique audience reactions, which need to be brought down

In [30]:
df_aud_clean = df_aud_react

In [31]:
# Change all column names to lower case
col_names = df_aud_clean.columns
col_names

Index(['Laughter', 'Applause', 'Mock sob',
       'Music: "The Sound of Silence," Simon & Garfunkel', 'Piano music',
       'Music', 'Audience whistles', 'Applause ends', 'Laughter and applause',
       'Applause continues',
       ...
       'Distorted voice', 'Speaks more loudly', 'Khmer', 'Singing in Arabic',
       'Cracking sound', 'Audience guesses', 'Audience murmurs',
       'Audience responds', 'A capella singing', 'offscreen'],
      dtype='object', length=791)

In [34]:
new_col_names = [X.lower() for X in col_names]
# new_col_names

In [35]:
df_aud_clean.columns = new_col_names

In [36]:
df_aud_clean.columns

Index(['applause', 'laughter', 'mock sob', 'music',
       'music: "the sound of silence," simon & garfunkel', 'piano music',
       'audience whistles', 'applause ends', 'laughter and applause',
       'applause continues',
       ...
       'distorted voice', 'speaks more loudly', 'khmer', 'singing in arabic',
       'audience guesses', 'cracking sound', 'audience murmurs',
       'audience responds', 'offscreen', 'a capella singing'],
      dtype='object', length=791)

In [37]:
# df_aud_clean.describe()

In [38]:
df_aud_clean.shape

(2412, 791)

In [39]:
df_aud_clean = df_aud_clean.dropna(axis = 1, thresh = 10)
df_aud_clean.shape

(2412, 12)

In [40]:
df_aud_clean.columns

Index(['applause', 'laughter', 'music', 'applause ends', 'cheering', 'sighs',
       'singing', 'video', 'music ends', 'laughs', 'audio', 'cheers'],
      dtype='object')

By dropping all columns which have less than 15 values, we drop the column count from 791 to 10. Lets look at the columns

In [41]:
df_aud_clean.describe()

,applause,laughter,music,applause ends,cheering,sighs,singing,video,music ends,laughs,audio,cheers
count,2288.000000,1767.000000,205.000000,27.000000,15.000000,10.000000,16.000000,167.000000,16.000000,39.000000,11.000000,28.000000
mean,2.291958,5.615733,2.780488,1.481481,1.133333,1.600000,2.500000,2.083832,2.562500,1.179487,5.090909,1.357143
std,1.987826,6.567969,2.951505,1.014145,0.351866,1.577621,2.804758,1.961722,1.672075,0.506370,5.769827,0.621485
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000
50%,1.000000,4.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.500000,1.000000,3.000000,1.000000
75%,3.000000,7.000000,3.000000,1.500000,1.000000,1.000000,2.250000,3.000000,3.250000,1.000000,4.500000,2.000000
max,18.000000,74.000000,20.000000,5.000000,2.000000,6.000000,9.000000,14.000000,7.000000,3.000000,17.000000,3.000000


Further consolidating the columns


In [42]:
df_aud_clean = df_aud_clean.fillna(0)
df_aud_clean.head()


,applause,laughter,music,applause ends,cheering,sighs,singing,video,music ends,laughs,audio,cheers
0,4.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12.0,38.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,10.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
df_aud_clean.shape

(2412, 12)

In [44]:
df_aud_clean['laughter'] = df_aud_clean['laughter'].add(df_aud_clean['laughs'])
df_aud_clean['applause'] = df_aud_clean['applause'].add(df_aud_clean['applause ends'])
df_aud_clean['music'] = df_aud_clean['music'].add(df_aud_clean['music ends'])
df_aud_clean['cheering'] = df_aud_clean['cheering'].add(df_aud_clean['cheers'])
df_aud_clean.describe()

,applause,laughter,music,applause ends,cheering,sighs,singing,video,music ends,laughs,audio,cheers
count,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000
mean,2.190713,4.133085,0.253317,0.016584,0.022803,0.006633,0.016584,0.144279,0.016998,0.019071,0.023217,0.015755
std,2.049802,6.154457,1.267634,0.188135,0.186373,0.140942,0.300240,0.738184,0.246336,0.161806,0.505753,0.159584
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,18.000000,74.000000,27.000000,5.000000,3.000000,6.000000,9.000000,14.000000,7.000000,3.000000,17.000000,3.000000


In [45]:
df_aud_clean.drop(columns = ['laughs', 'applause ends', 'music ends','cheers'], inplace = True)


In [46]:
df_aud_clean.shape

(2412, 8)

In [47]:
df_clean.shape

(2412, 17)

In [48]:
df_aud_clean.columns

Index(['applause', 'laughter', 'music', 'cheering', 'sighs', 'singing',
       'video', 'audio'],
      dtype='object')

In [49]:
df_clean = pd.merge (df_clean, df_aud_clean, how = 'inner', left_index = True, right_index = True)
df_clean.shape

(2412, 25)

## Extract Ratings as columns in df

In [50]:
# See an example element in ratings column
ratings_example = df_clean.loc[2,'ratings']
ratings_example

"[{'id': 7, 'name': 'Funny', 'count': 964}, {'id': 3, 'name': 'Courageous', 'count': 45}, {'id': 9, 'name': 'Ingenious', 'count': 183}, {'id': 1, 'name': 'Beautiful', 'count': 60}, {'id': 21, 'name': 'Unconvincing', 'count': 104}, {'id': 11, 'name': 'Longwinded', 'count': 78}, {'id': 8, 'name': 'Informative', 'count': 395}, {'id': 10, 'name': 'Inspiring', 'count': 230}, {'id': 22, 'name': 'Fascinating', 'count': 166}, {'id': 2, 'name': 'Confusing', 'count': 27}, {'id': 25, 'name': 'OK', 'count': 146}, {'id': 24, 'name': 'Persuasive', 'count': 230}, {'id': 23, 'name': 'Jaw-dropping', 'count': 54}, {'id': 26, 'name': 'Obnoxious', 'count': 142}]"

In [51]:
type(ratings_example)

str

In [52]:
RatingsDict(ratings_example)

{'Funny': 964,
 'Courageous': 45,
 'Ingenious': 183,
 'Beautiful': 60,
 'Unconvincing': 104,
 'Longwinded': 78,
 'Informative': 395,
 'Inspiring': 230,
 'Fascinating': 166,
 'Confusing': 27,
 'OK': 146,
 'Persuasive': 230,
 'Jaw-dropping': 54,
 'Obnoxious': 142}

In [53]:
df_clean['ratings_dict'] = df_clean['ratings'].apply(RatingsDict)

In [54]:
df_clean.head()

,comments,description,duration,event,film_date,main_speaker,published_date,ratings,speaker_occupation,tags,...,word_count,applause,laughter,music,cheering,sighs,singing,video,audio,ratings_dict
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,Ken Robinson,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",...,3172,4.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'Funny': 19645, 'Beautiful': 4573, 'Ingenious..."
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,Al Gore,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...",Climate advocate,"['alternative energy', 'cars', 'climate change...",...,2081,6.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'Funny': 544, 'Courageous': 139, 'Confusing':..."
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,David Pogue,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",...,3375,12.0,38.0,2.0,0.0,0.0,0.0,0.0,0.0,"{'Funny': 964, 'Courageous': 45, 'Ingenious': ..."
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,Majora Carter,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",...,3174,7.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'Courageous': 760, 'Beautiful': 291, 'Confusi..."
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,Hans Rosling,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",...,3343,4.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'Ingenious': 3202, 'Funny': 1390, 'Beautiful'..."


In [55]:
df_ratings = df_clean['ratings_dict'].apply(pd.Series)
df_ratings.head()

,Funny,Beautiful,Ingenious,Courageous,Longwinded,Confusing,Informative,Fascinating,Unconvincing,Persuasive,Jaw-dropping,OK,Obnoxious,Inspiring
0,19645,4573,6073,3253,387,242,7346,10581,300,10704,4439,1174,209,24924
1,544,58,56,139,113,62,443,132,258,268,116,203,131,413
2,964,60,183,45,78,27,395,166,104,230,54,146,142,230
3,59,291,105,760,53,32,380,132,36,460,230,85,35,1070
4,1390,942,3202,318,110,72,5433,4606,67,2542,3736,248,61,2893


In [56]:
df_ratings.shape

(2412, 14)

In [57]:
df_ratings.describe()

,Funny,Beautiful,Ingenious,Courageous,Longwinded,Confusing,Informative,Fascinating,Unconvincing,Persuasive,Jaw-dropping,OK,Obnoxious,Inspiring
count,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000,2412.000000
mean,156.647181,191.935323,152.349917,168.856136,33.152985,21.152570,358.137231,319.441542,53.824212,226.013682,148.414179,82.154229,25.851575,537.966003
std,603.386141,482.028172,285.298216,440.837916,42.178191,32.083401,550.546472,634.477075,93.649344,475.045306,562.084608,90.329758,53.764271,1305.633290
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
25%,8.000000,26.000000,27.000000,20.000000,9.000000,5.000000,106.000000,81.000000,12.000000,39.000000,18.000000,32.000000,6.000000,115.000000
50%,21.000000,67.000000,70.000000,53.000000,19.000000,12.000000,220.000000,164.000000,27.000000,100.000000,42.000000,57.000000,13.000000,238.000000
75%,91.000000,184.000000,172.000000,156.000000,41.000000,24.000000,411.000000,351.250000,60.250000,233.250000,110.000000,101.000000,27.000000,505.250000
max,19645.000000,9437.000000,6073.000000,8668.000000,447.000000,531.000000,9787.000000,14447.000000,2194.000000,10704.000000,14728.000000,1341.000000,1361.000000,24924.000000


In [58]:
df_ratings.columns

Index(['Funny', 'Beautiful', 'Ingenious', 'Courageous', 'Longwinded',
       'Confusing', 'Informative', 'Fascinating', 'Unconvincing', 'Persuasive',
       'Jaw-dropping', 'OK', 'Obnoxious', 'Inspiring'],
      dtype='object')

In [59]:
# Add ratings to the df_clean
df_clean = pd.merge (df_clean, df_ratings, how = 'inner', left_index = True, right_index = True)
df_clean.head()

,comments,description,duration,event,film_date,main_speaker,published_date,ratings,speaker_occupation,tags,...,Longwinded,Confusing,Informative,Fascinating,Unconvincing,Persuasive,Jaw-dropping,OK,Obnoxious,Inspiring
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,Ken Robinson,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",...,387,242,7346,10581,300,10704,4439,1174,209,24924
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,Al Gore,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...",Climate advocate,"['alternative energy', 'cars', 'climate change...",...,113,62,443,132,258,268,116,203,131,413
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,David Pogue,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",...,78,27,395,166,104,230,54,146,142,230
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,Majora Carter,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",...,53,32,380,132,36,460,230,85,35,1070
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,Hans Rosling,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",...,110,72,5433,4606,67,2542,3736,248,61,2893


In [60]:
df_clean.shape

(2412, 40)

# Write Cleaned Dataset to File

In [61]:
# Add column for event type based on event
df_clean['event_type'] = df_clean['event'].apply(ted_eventtype)

In [63]:
# df_clean.head()

In [62]:
df_clean.to_csv('../data/interim/After_DataWrang_Out.csv')